# **Chapter 8 - Dimensionality Reduction**
Many Machine Learning Problems Involve thousands or even hundreds of features to Train the Models which In turn gives birth to **the curse of dimensionality**(Explained Later in the Chapter) and as a solution to this Problem comes Dimensionality Reduction here are two of the advantages of using Dimensionality Reduction :-
- Speed up the Process of Training.
- Helpful in Data Visualization.

# The Curse of Dimensionality
- Sometimes Our Machine Learning Model gets confused with the Noise of the Dataset.
    - Most of the times due to high number of features.
- High Numbers of features in a dataset results in slow speed of Training.

# Main Approaches of Dimensionality Reduction
There are two main Approaches of Dimensionality Reduction in Machine Learning :-

## Projection
In Most Real World Problems the the Dataset is not Uniformly Distrubuted Throughout all Dimensions. Many Features are almost Constant.
So in the Projection We Find out the Plane in which most of the data points lie and then Project the Whole Dataset on that specific Plane.
However somtimes we cannot find a plane on the Dataset Like in the famous Swiss Roll Dataset in Which You have to unroll the dataset to get accurate representation.

## Manifold Learning
Manifold Learning is a Dimensionality Reduction Algorithm which Tries to find Familiar Shapes or Structures inside the Dataset and then Projects the Dataset Unto a Lower Dimensional Plane hence Reducing its Dimension.
- i.e > Unrolling the Swiss Roll Dataset.

# PCA(Principal Component Analysis)
- This is the Most Popular Dimensionality Redution Algorithm.
- First It Identifies the Hyperplane which Lies closest to the Data.
- Then It Projects the Data Onto that Hyperplane.

## Preserving the Variance
- Before You Project the Data onto the Hyperplane You Need to Select the Right Hyperplane.
- The Best Way to Select the Right Hyperplane is to choose the Hyperplane which Preserves the Maximum Variance.
    - Because it will in turn preserve the Data Loss.

## Principal Component
- PCA Identifies the axis that accounts the largest amount of Variance in the training Set.
- The $i^{th}$ axis is also called the $i^{th}$ *principal component*.
- We find the principal component for the dataset with help of a standard matrix factorization called **Singular Value Decomposition(SVD)** here is an implementation of this function with python code :-

In [1]:
from sklearn.datasets import make_moons
X, y = make_moons()

In [2]:
import numpy as np

X_centered = X - X.mean(axis=0)
U, s, Vt = np.linalg.svd(X_centered)
c1 = Vt.T[:, 0]
c2 = Vt.T[:, 1]

**V** contains all of the Pricipal Components that we are looking for :-
$$
V = (c_1, c_2, ..., c_n)
$$
- here vectors from $c_1$ to $c_n$ are the Pricipal Components from which main Pricipal Componenet will be choosed on the basis of Preserving Variance.

## Projecting Down to d Dimensions
Now that we have Identified our Pricipal Component we can obtain the reduced data by Prjecting it onto the Pricipal component of our choice.
- To Project the training set onto the hyperplane and obtain a reduced dataset $X_{d-proj}$ we have to compute a matrix multiplication.

$$
X_{d-proj} = XW_d
$$

In [3]:
W2 = Vt.T[:, :2]
X2D = X_centered.dot(W2)

## Using Scikit-Learn
Like all the other Techniques and other things *Scikit-Learn* also Provides an implementation of **PCA** :-

In [4]:
from sklearn.decomposition import PCA

pca = PCA(n_components=2)
X2D = pca.fit_transform(X)

In [5]:
pca.explained_variance_ratio_

array([0.81968748, 0.18031252])

This tells YOu that the VAriance in the first PC was 81% and in the Second PC it was 18% which means that we have Preserved 81% of Our Data.

## Choosing the Right Number of Dimensions
Instead of Choosing the Number of Dimensions by mere Guess we can Set the Percentage of the Data that we want to Preserve from a scale between 0 to 1 in the **n_components** hyperparameter. Lets apply it on the MNIST Dataset.

In [6]:
from sklearn.datasets import fetch_openml

mnist = fetch_openml("mnist_784", version=1)
X, y = mnist["data"], mnist["target"]

pca = PCA(n_components=0.95) # Preserve 95% Data of the Dataset
X_reduced = pca.fit_transform(X)
X_reduced

/home/praddyumny/MachineLearning/Hands_On_Machine_Learning/venv/lib64/python3.12/site-packages/sklearn/datasets/_openml.py:1022: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


array([[ 122.25525533, -316.23384391,  -51.13183087, ...,   34.71703473,
         -14.22575676,   21.38272145],
       [1010.49400346, -289.96362059,  576.1207452 , ...,   23.87884359,
          -6.54283564,  -24.90277545],
       [ -58.99594719,  393.69744499, -161.99818411, ...,   -5.36282742,
          55.00020853,  -96.73397123],
       ...,
       [-271.50701323,  590.07850009,  341.36886918, ...,  -43.7571469 ,
          35.78216024,   49.96612771],
       [-310.22482291, -116.72715081,  635.71999693, ...,  -21.86345345,
          20.40152778,  -42.68277473],
       [1058.86212574,  -83.39253843,  731.34218396, ...,   41.22834049,
         -20.05206663,  -49.92361814]])

In [7]:
X_reduced.shape

(70000, 154)

## PCA for Compression
We can see that after applying PCA to MNIST it Preserves 95% Variance and the Dataset is now Less 20% BTW we can also recover the original dataset (however it it will not be the exact same some of the things can be different)

In [8]:
X_recovered = pca.inverse_transform(X_reduced)
X_recovered.shape

(70000, 784)

$$
X_{recovered} = X_{d-proj} . W_d^T
$$
**--------------------------------------------- Equation 8-3: PCA inverse transformation ---------------------------------------------**

## Randomized PCA
If you set *svd_solver* hyperparameter to "randomized" Scikit-Learn uses Stochastic Algorithm called **Randomized PCA**.

In [9]:
rnd_pca = PCA(n_components=154, svd_solver="randomized")
X_reduced = rnd_pca.fit_transform(X)
X_reduced.shape

(70000, 154)

## Incremental PCA
One problem with PCA is that they require whole dataset to fit in the memory but luckily we can avoid it by using **Incremental PCA(IPCA)** which splits the datasets in mini-batches and this is a good way to upgrade to online learning.

In [10]:
from sklearn.decomposition import IncrementalPCA

n_batches = 100
inc_pca = IncrementalPCA(n_components=154)
for X_batch in np.array_split(X, n_batches):
    inc_pca.partial_fit(X_batch)

X_reduced = inc_pca.transform(X)
X_reduced.shape

/home/praddyumny/MachineLearning/Hands_On_Machine_Learning/venv/lib64/python3.12/site-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


(70000, 154)

# Kernel PCA
In Chapter 5 we discussed about a Kernel Trick a methematical technique that implicitly maps instances into a higher dimensional space called feature space enabling non-linear classification and regression with Support Vector Machines(SVMs).
It turns Out that the same trick can be applied to PCA makeing it possible to project complex non-linear projections for dimensionality reduction.

In [11]:
from sklearn.datasets import make_swiss_roll
X, color = make_swiss_roll()
X.shape

(100, 3)

In [12]:
from sklearn.decomposition import KernelPCA
rbf_pca = KernelPCA(n_components=2, kernel="rbf", gamma=0.04)
X_reduced = rbf_pca.fit_transform(X)
X_reduced.shape

(100, 2)

## Selecting and Tuning Hyperparameters
We know that Kernel PCA is an Unsupervised Learning Algorithm so there isn't any obvious performance measure for it but it is kind of a preparation step for a supervised learning task so that means we can use GridSearchCV to choose the kernel which leads to the best performance on that particular task.

In [13]:
X, y = make_moons()

In [14]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline

clf = Pipeline([
    ("kpca", KernelPCA(n_components=2)),
    ("log_reg", LogisticRegression())
])

param_grid = [{
    "kpca__gamma": np.linspace(0.03, 0.05, 10),
    "kpca__kernel": ["rbf", "sigmoid"]
}]

grid_search = GridSearchCV(clf, param_grid, cv=3)
grid_search.fit(X, y)

GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('kpca', KernelPCA(n_components=2)),
                                       ('log_reg', LogisticRegression())]),
             param_grid=[{'kpca__gamma': array([0.03      , 0.03222222, 0.03444444, 0.03666667, 0.03888889,
       0.04111111, 0.04333333, 0.04555556, 0.04777778, 0.05      ]),
                          'kpca__kernel': ['rbf', 'sigmoid']}])

In [15]:
grid_search.best_params_

{'kpca__gamma': 0.03, 'kpca__kernel': 'rbf'}

# LLE - Locally Linear Embedding
This is a Dimensionality Reduction Algorithm which takes a very different Path in Comparison with PCA and Many others so Basically what it does is that it finds the relationship between every instance and its closest instances then it reduces the dimensions of it and then it tries to re-establish the relationship between them here is an implementation from scikit-learn :-

In [16]:
from sklearn.manifold import LocallyLinearEmbedding

lle = LocallyLinearEmbedding(n_components=2, n_neighbors=10)
X_reduced = lle.fit_transform(X)

In [17]:
X_reduced

array([[-0.0240951 ,  0.0917334 ],
       [ 0.15128302, -0.12505169],
       [ 0.1697423 , -0.17336364],
       [-0.16974231, -0.17336364],
       [-0.06184254,  0.06126423],
       [-0.02881438,  0.08971436],
       [-0.17425115, -0.18522388],
       [ 0.04768922,  0.07656466],
       [-0.183022  , -0.20834723],
       [-0.11841504, -0.04321621],
       [ 0.10428083, -0.011587  ],
       [ 0.18726189, -0.21952952],
       [-0.09013917,  0.01681707],
       [ 0.06655963,  0.05511734],
       [-0.01465651,  0.0943087 ],
       [ 0.1372327 , -0.08904099],
       [-0.01937563,  0.09326113],
       [ 0.0571251 ,  0.06688794],
       [-0.01829187,  0.08832706],
       [-0.0276747 ,  0.08233493],
       [-0.09485365,  0.00775414],
       [-0.0099394 ,  0.09488361],
       [-0.14192441, -0.10093727],
       [ 0.12782966, -0.06570539],
       [-0.11370462, -0.03236172],
       [ 0.08070863,  0.03360242],
       [ 0.01829186,  0.08832706],
       [-0.03653287,  0.07584864],
       [-0.04162802,

This algotithm particularly works better on the datasets like swiss roll dataset.

# Other Dimensionlaity Reduction Techniques
There are So many Dimensionality Reduction Techniques in Scikit-Learn Here are some of the most important ones :-
## Random Projections
As its name suggests it just Projects the Dataset Randomly to a Lower Dimensional Plane, Well! It sounds crazy but there is a very high possibility that it preserves the distances very well (Proved by **William B. Johnson** and **Joran Lindenstrauss**).
## Multi-Dimensional Scaling (MDS)
Reduces dimensionality while trying to preserve the distances between the instances.
## Isomap
Measures Distance between Closest Instances then it tries to Preserve the Distances between them while reducing Dimensions.
## t-Distributed Stochastic Neighbor Embedding (t-SNE)
Tries to Reduce the Dimensions while trying to Close the Distance Between Similar Objects and Increase the Distance between Disssimilar Objects (It is mainly used for Visualization Purposes).
## Linear Discriminant Analysis (LDA)
It is a Classification Algorithm so it learns the most discriminative axes then it can be used to create a Hyperplane on which the dataset is yet to be projected It is a good warm up algorithm it is often Good to run it on a dataset before training other Classifier Like SVM Classfier.